# Comparing implemented methods
## Imports

In [84]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

## Loading [SFU's surface reflectance dataset](https://www2.cs.sfu.ca/~colour/data/colour_constancy_synthetic_test_data/)
Sorry for magic numbers. We need equal ranges of data that's why we skip some fields
sometimes.

In [85]:
sfu_spd = np.loadtxt("data/reflect_db.reflect", dtype=np.float64).reshape([-1,101])[:, 5:86:]
data_size = sfu_spd.shape[0]

interp_wavelengths = np.arange(400, 721, 2)
interpolator = make_interp_spline(interp_wavelengths[::2], sfu_spd, k=1, axis=1)
sfu_spd = interpolator(interp_wavelengths).reshape(data_size, -1, 1)

sfu_spd.shape

(1995, 161, 1)

## Loading (Nikon D5100 spectral sensivity function)[https://github.com/butcherg/ssf-data/blob/master/Nikon/D5100/camspec/Nikon_D5100.csv]

In [86]:
nikon_ssf = np.loadtxt("data/Nikon_D5100.csv", dtype=np.float64, delimiter=",", usecols=(1,2,3))

interpolator = make_interp_spline(interp_wavelengths[::5], nikon_ssf, k=1, axis=0)
nikon_ssf = interpolator(interp_wavelengths)

nikon_ssf.shape

(161, 3)

## Loading (XYZ color matching function)[https://cie.co.at/datatable/cie-1931-colour-matching-functions-2-degree-observer]

In [87]:
xyz_cmf = np.loadtxt("data/CIE_xyz_1931_2deg.csv", dtype=np.float64, delimiter=",", usecols=(1,2,3))[40:361:2]
xyz_cmf.shape

(161, 3)

## Loading [D65 spectral poower distribution](https://cie.co.at/datatable/cie-standard-illuminant-d65)

In [94]:
d65_spd = np.loadtxt("data/CIE_std_illum_D65.csv", dtype=np.float64, delimiter=",", usecols=(1))[100:421:2].reshape(-1, 1)
d65_spd.shape

(161, 1)

## Calculating RGB RAW and CIE XYZ coordinates

In [95]:
rgb = np.trapz(sfu_spd * nikon_ssf * d65_spd, dx=2, axis=1)
xyz = np.trapz(sfu_spd * xyz_cmf * d65_spd, dx=2, axis=1)
print(rgb.shape, xyz.shape)

(1995, 3) (1995, 3)
